In [0]:
%sql
WITH produtos AS (
    SELECT
        codigo,
        nome,
        origem

    FROM nintendodatabrickswi86no_workspace.gold.products
),
preco AS (
    SELECT
        codigo,
        preco
    FROM nintendodatabrickswi86no_workspace.gold.price
),

detalhes AS (
    SELECT
        codigo,
        oled,
        link,
        recomendacao
    FROM nintendodatabrickswi86no_workspace.gold.details
),

produtos_filtrados AS (
    SELECT
        produtos.codigo,
        produtos.nome,
        produtos.origem,
        preco.preco,
        detalhes.oled,
        detalhes.link,
        detalhes.recomendacao
    FROM produtos
    JOIN preco ON produtos.codigo = preco.codigo
    JOIN detalhes ON produtos.codigo = detalhes.codigo
    WHERE detalhes.oled = 'Sim' AND produtos.origem IN (SELECT DISTINCT origem FROM nintendodatabrickswi86no_workspace.gold.products)
),

menores_precos AS (
    SELECT
        -- Macaramento da coluna Codigo inserindo '*' entre as informações
        CONCAT(SUBSTR(codigo, 1, 2), REPEAT('*', LENGTH(codigo) - 4), SUBSTR(codigo, -2)) AS codigo,
        preco AS menor_preco,
        recomendacao,
        origem,
        -- Mascaramento do link obtendo tudo que está antes do '.com' para proteger o codigo do produto
        REGEXP_EXTRACT(link, '^(.*?\\.com)') AS link

    FROM produtos_filtrados
    QUALIFY ROW_NUMBER() OVER (PARTITION BY origem ORDER BY preco ASC) = 1

)

SELECT * FROM menores_precos